In [6]:
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://lumin-azure-openai-rnd.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "38002279fa574fbcba0fbc424ae056d5"

In [7]:
import os
from openai import AzureOpenAI

In [8]:
client = AzureOpenAI(
  azure_endpoint = "https://lumin-azure-openai-rnd.openai.azure.com/", 
  api_key="38002279fa574fbcba0fbc424ae056d5",  
  api_version="2023-03-15-preview"
)